#### 基于生成器的协程

In [1]:
# 对比 generator 和 coroutine

def generator_func():
    yield

gen = generator_func()

print(gen)
print(sorted(set(dir(gen)) - set(dir(object))))

<generator object generator_func at 0x000002A99E2F9AF0>
['__del__', '__iter__', '__name__', '__next__', '__qualname__', 'close', 'gi_code', 'gi_frame', 'gi_running', 'gi_yieldfrom', 'send', 'throw']


In [2]:
async def coroutine_func():
    await coroutine_func()

coro = coroutine_func()

print(coro)
print(sorted(set(dir(coro)) - set(dir(object))))

<coroutine object coroutine_func at 0x000002A99E2FA3B0>
['__await__', '__del__', '__name__', '__qualname__', 'close', 'cr_await', 'cr_code', 'cr_frame', 'cr_origin', 'cr_running', 'send', 'throw']


In [3]:
""" yield 表达式
PEP 342 -- Coroutines via Enhanced Generators把 yield 关键字升级成了表达式。
所谓表达式（Expression），意味着它可以被解析成一个值。
同时，yield语句的特性依然保留，不能出现在顶层代码中，必须在函数体内使用。
"""
# 赋值给变量
x = yield

# 计算后赋值给变量
y = yield + 1

# yield 可以作为函数入参，但是需要使用()括起来
print((yield))

SyntaxError: 'yield' outside function (1574197157.py, line 7)

In [4]:
def show_yield_value():
    x = yield
    print(f'x is {x}')

g = show_yield_value()
next(g)  # 第1次

next(g)  # 第2次

# 使用 next() 函数来驱动生成器的时候，yield 表达式的值总是为 None。

x is None


StopIteration: 

In [5]:
"""
为生成器增加一个 send() 方法，该方法可以接受一个入参。
send 方法顾名思义，将该参数发送给生成器，使生成器恢复运行的同时，将该入参作为 yield表达式的值。
"""
def show_yield_value():
    print('开始')
    x = yield
    print(f'x is {x}')

g = show_yield_value()
g.send('hello')  # 第 1 次只能是 None

TypeError: can't send non-None value to a just-started generator

In [ ]:
"""
prime
对于刚创建好的生成器，总是需要在第一次的时候 send(None) 值，使其运行到 yield 的地方暂停，这个步骤术语称为 prime。
"""

g.send(None)

开始


In [7]:
g.send('hello')  # 第 2 次将 'hello' 作为 yield 表达式的值

x is hello


StopIteration: 

#### yield 表达式的优先级

In [ ]:
def add_yield_value():
    x = yield + 1  # bug! 等价于 x = (yield 1)
    print(f'x is {x}')

g = add_yield_value()
g.send(None)  # prime

1

In [ ]:
g.send(3)


x is 3


StopIteration: 

In [ ]:
def add_yield_value():
    x = (yield) + 1  
    print(f'x is {x}')

g = add_yield_value()
g.send(None)  # prime

g.send(3)

x is 4


StopIteration: 

##### send() 用法总结
- send 是生成器对象的方法
- 对于生成器对象 g，next(g) 等价于 g.send(None)
- 只有当生成器处在「暂停」状态时，才能传入非None的值
- send 方法是为了协程而增加的API，所以：
    - 如果将生成器视作协程，就应该只用 send 方法
    - 如果视作迭代器，就仍用 next

#### 一个极简的 echo

In [ ]:
def gen_echo():
    while True:
        print((yield))

echo = gen_echo()
echo.send(None)  # prime

for action in ["hello", "world", 42]:
    echo.send(action)


echo.send(StopIteration('stop')) # StopIteration 作为普通值传入，不会终止生成器

hello
world
42
stop


In [ ]:
"""
使用 close() 结束生成器
当生成器作为迭代器来用的时候，它的生命周期取决于有多少元素可以迭代。
而当作协程来用的时候，通常可以视作是在执行一个任务，新增的 close 方法就是用来结束一个协程。
"""
echo.close()

echo.send('hi')

StopIteration: 

In [ ]:
"""
由于 echo 协程的内容非常简单，所以可以直接结束。
如果协程的代码比较复杂，它可能需要在结束的时候做一些善后处理，比如释放资源等。
类似于 StopIteration 的实现机制，结束协程也是靠异常来实现的：
"""
def gen_echo_v2():
    while True:
        try:
            x = yield
        except GeneratorExit:
            print('exit. bye!')
            return  # 处理完异常后，必须要跳出循环
        else:
            print(x)

In [ ]:
echo_v2 = gen_echo_v2()
echo_v2.send(None)  # 需要激活才能捕获到异常
echo_v2.close()

exit. bye!


In [ ]:
# 除了显式地调用 close 方法，如果生成器对象被垃圾回收，也会自动调用 close：
echo_v2 = gen_echo_v2()
echo_v2.send(None)  # 需要激活才能捕获到异常

# del echo_v2

echo_v2 = 123

exit. bye!


#### 使用 throw() 将异常抛给 yield

In [ ]:
def gen_echo_v3():
    while True:
        try:
            x = yield
        except GeneratorExit:
            print('exit. bye!')
            return
        except KeyboardInterrupt:
            print('Ctrl-C ')
        else:
            print(x)

In [ ]:
echo_v3 = gen_echo_v3()
echo_v3.send(None) # prime 

In [ ]:
echo_v3.throw(KeyboardInterrupt)  # 抛出异常

Ctrl-C 
